In [1]:
import os
import numpy as np
import pandas as pd
import math

In [2]:
# # export
# def synthesize_speakerids(filelists):

#     data_dict = {}
#     for f in range(len(filelists)):
#         data_dict[filelists[f]] = pd.read_csv(filelists[f], sep = ",",index_col=0,error_bad_lines=False)
#         #pd.read_csv(filelists[f], sep = "|",header=None, error_bad_lines=False)
        
#     source_files = list(data_dict.keys())
#     nspeakers_cumulative = 0
#     speaker_offset = {}
#     for source_file in filelists:
#         data = data_dict[source_file]
#         nspeakers = len(np.unique(data.iloc[:,2]))
#         data.iloc[:,2] = data.iloc[:,2] + nspeakers_cumulative
#         data_dict[source_file] = data
#         speaker_offset[source_file] = nspeakers_cumulative
#         nspeakers_cumulative = nspeakers_cumulative + nspeakers

#     return(data_dict)  

In [3]:
# export
import soundfile as sf


def load_filepaths_and_text(filename, split="|"):
    with open(filename, encoding='utf-8') as f:
        filepaths_and_text = [line.strip().split(split) for line in f]
    return filepaths_and_text

def synthesize_speakerids2(filelists, fix_indices_index = None):

    data_dict = {}
    data_dict_out = {}
    for f in range(len(filelists)):
        #data_dict[filelists[f]] = pd.read_csv(filelists[f], sep = ",",index_col=0,error_bad_lines=False)
            data = load_filepaths_and_text(filelists[f])
            data_dict[filelists[f]] = pd.DataFrame(data)
            data_dict[filelists[f]].iloc[:,2] = np.asarray(data_dict[filelists[f]].iloc[:,2], dtype = int)
            data_dict[filelists[f]].iloc[:,2] = np.asarray(data_dict[filelists[f]].iloc[:,2], dtype = object)
        #pd.read_csv(filelists[f], sep = "|",header=None, error_bad_lines=False)
        
    source_files = list(data_dict.keys())
    
    speaker_offset = {}
    nfilelist = len(filelists)
    reserved_speakers = np.asarray(np.unique(data_dict[filelists[fix_indices_index]].iloc[:,2]), dtype = object)
    
    for s in range(nfilelist):
        source_file = filelists[s]
        #print(s,source_file)
        data = data_dict[source_file]
        if s != fix_indices_index:
            speakers = np.asarray(np.unique(data.iloc[:,2]), dtype = object)
            #print(speakers,'speaker_input')
            overlap = np.where(np.isin(speakers, reserved_speakers))[0]
            reserved_speakers_temp = np.union1d(speakers, reserved_speakers)
            naive_guess = np.asarray(list(range(len(reserved_speakers) + len(speakers))), dtype = object)
            newindices = np.setdiff1d(naive_guess, reserved_speakers_temp)[:len(overlap)]
            #print(s,len(overlap))
            for o in range(len(overlap)):
                data.iloc[np.where(data.iloc[:,2] == overlap[o])[0] ,2] = newindices[o]
            #print(newindices,'new_index')
            data_dict_out[source_file] = data
            speakers = np.unique(data.iloc[:,2])
            reserved_speakers = np.union1d(speakers, reserved_speakers)
            #print(speakers,reserved_speakers)
        else:
            data_dict_out[source_file] = data
            
    return(data_dict_out)  

In [4]:
# SAMPLE_RATE = 22050
# loader = Loader(sample_rate=SAMPLE_RATE)
# padder = Padder(sample_rate=SAMPLE_RATE, pad_ms=50)
# trimmer = TrimSilence(leading_silence_threshold=-33, trailing_silence_threshold=-36)
# # normalizer = Normalizer(min_val=0,max_val=1)

# pipeline = PreprocessingPipeline(loader=loader, 
#                                    padder=padder, 
#                                    trimmer=trimmer)
# In [14]:
# input_folder = "vctk_sample_test/*/*.wav"
# output_folder = "vctk_sample_preprocessed"

# # Preprocess files in input folder and write to output folder


In [5]:
metalist_dir = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/metadata_collections'
metalist_files = os.listdir(metalist_dir)
print(metalist_files)
train_ratios = np.ones(4)*1.

['lj7_processed_file.npy', 'libritts_processed_file.npy', 'uberduck_processed_files.npy', 'vctk_processed_file.npy']


In [6]:
filelists=np.asarray([])
#files = np.asarray([])
for r in range(4):
    files = np.load(metalist_dir +'/'+ metalist_files[r],allow_pickle=True)
    filelist = np.asarray([])
    
    if files.ndim > 0:
        nfiles = files.shape[0]
        for s in range(nfiles):
            filelist = np.append(filelist, files[s])
    else: 
        filelist = files
    filelists = np.append(filelists, filelist)

In [7]:
print(filelists)
dd = synthesize_speakerids2(filelists,1)

['/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/processed_metadata/lj7all_processed.txt'
 '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/processed_metadata/librittsall_processed.txt'
 '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/processed_metadata/eminem_all_processed.txt'
 '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/processed_metadata/ben-shapiro_all_processed.txt'
 '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/processed_metadata/jay-z_all_processed.txt'
 '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/processed_metadata/vctkall_processed.txt']


In [8]:
ad = list(dd.values())
#ad2 = [ad[i] for i in [0,2,3]]
#ad2 = [ad[i] for i in [2,3,4]]
#ad2 = [ad[i] for i in [0,2,3,4]]
#ad2 = [ad[i] for i in [0,2,3,4]]
#ad2 = [ad[i] for i in [2,3,4]]
ad2 = [ad[i] for i in [2,3,4]]
#ad2 = [ad[i] for i in [0]]
alldata = pd.concat(ad2)

In [9]:
train_ind = np.random.choice(alldata.shape[0], math.floor(alldata.shape[0]*.8), replace = False)
val_ind = np.setdiff1d(list(range(alldata.shape[0])), train_ind)
alldata.iloc[train_ind,:].to_csv('/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_train.txt', header = False,sep = '|', index=False)
alldata.iloc[val_ind].to_csv('/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_val.txt', header = False,sep = '|', index=False)

In [9]:
train_ind = np.random.choice(alldata.shape[0], math.floor(alldata.shape[0]*.8), replace = False)
val_ind = np.setdiff1d(list(range(alldata.shape[0])), train_ind)
alldata.iloc[train_ind,:].to_csv('/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/em_train.txt', header = False,sep = '|', index=False)
alldata.iloc[val_ind].to_csv('/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/em_val.txt', header = False,sep = '|', index=False)

In [9]:
train_ind = np.random.choice(alldata.shape[0], math.floor(alldata.shape[0]*.8), replace = False)
val_ind = np.setdiff1d(list(range(alldata.shape[0])), train_ind)
alldata.iloc[train_ind,:].to_csv('/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/uber3_train.txt', header = False,sep = '|', index=False)
alldata.iloc[val_ind].to_csv('/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/uber3_val.txt', header = False,sep = '|', index=False)

In [22]:
SAMPLE_RATE = 22050
loader = Loader(sample_rate=SAMPLE_RATE)
padder = Padder(sample_rate=SAMPLE_RATE, pad_ms=50)
trimmer = TrimSilence(leading_silence_threshold=-33, trailing_silence_threshold=-36)
# normalizer = Normalizer(min_val=0,max_val=1)

pipeline = PreprocessingPipeline(loader=loader, 
                                   padder=padder, 
                                   trimmer=trimmer)
i = 0
for audio_file in alldata.iloc[:2,0]:
    i = i+1
    print(i)
    # Grab filename and create path to file in new location
    path_parts = audio_file.split(".wav")
    file_name = path_parts[-2]
#    output_path = os.path.join(output_folder,path_parts[1])
#     if not os.path.exists(output_path):
#         os.mkdir(output_path)
        
#     # Run data through pipeline
    
    original, processed = pipeline.preprocess(audio_file)
    output_path = file_name + "_processed.wav"
    #sf.write(output_path,processed,samplerate=22050)
#     # Write new data
#     write_path = audio_file #os.path.join(output_path,file_name)
#     librosa.output.write_wav(write_path,processed,sr=SAMPLE_RATE)

NameError: name 'Loader' is not defined

In [29]:
import IPython
import soundfile as sf
IPython.display.Audio("/mnt/disks/uberduck-experiments-v0/data/uberduck/eminem/wav/01.wav")

In [46]:
import sounddevice as sd

fs = 22050
data = np.random.uniform(-1, 1, fs)
sd.play(processed, fs)

OSError: PortAudio library not found

In [30]:
# for i in range(alldata.shape[0]):
#     path_parts = alldata.iloc[i,0].split(".wav")
#     file_name = path_parts[-2]
#     output_path = file_name + "_processed.wav"
#     alldata.iloc[i,0] = output_path

In [32]:
alldata.iloc[0,0]

'/mnt/disks/uberduck-experiments-v0/data/uberduck/eminem/wav/01.wav'

In [13]:
    def create_speaker_lookup_table(audiopaths_and_text):
        speaker_ids = np.sort(np.unique([x[2] for x in audiopaths_and_text]))
        d = {int(speaker_ids[i]): i for i in range(len(speaker_ids))}
        return d

In [19]:
#audiopaths_and_text = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/todos_train.txt'
audiopaths_and_text = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/todos_train.txt'
audiopaths_and_text = load_filepaths_and_text(audiopaths_and_text)
#asdf = [x[2] for x in audiopaths_and_text]

In [35]:
from numba import cuda 
cuda.select_device(0)
cuda.close()

In [20]:
create_speaker_lookup_table(audiopaths_and_text)

{0: 0}

In [242]:
import librosa

In [244]:
librosa.load(audiopaths_and_text[4][0])

(array([ 0.00335953,  0.00497484,  0.00431533, ..., -0.00397925,
        -0.00451155,  0.        ], dtype=float32),
 22050)

In [245]:
sf.read(audiopaths_and_text[4][0])

(array([ 0.00469971,  0.00460815,  0.00466919, ..., -0.00439453,
        -0.00415039, -0.00308228]),
 48000)

In [239]:
audiopaths_and_text[5][0]

'/mnt/disks/uberduck-experiments-v0/data/LJSpeech-1.1/wavs/LJ010-0305.wav'

In [154]:
filename = audiopaths_and_text

In [155]:
with open(filename, encoding='utf-8') as f:
    filepaths_and_text = [line for line in f]

In [ ]:
### source_files = [experiment_filelist_dir + 'eminem_all.txt',
#             '/mnt/disk/uberduck-experiments-v0/data/uberduck/kanye-rap/all.txt',
#             '/mnt/disk/uberduck-experiments-v0/data/uberduck/alex-trebek/all.txt',
#             '/mnt/disk/uberduck-experiments-v0/data/uberduck/ben-shapiro/all.txt',
#             '/mnt/disk/uberduck-experiments-v0/data/uberduck/michael-rosen/all.txt',
#             '/mnt/disk/uberduck-experiments-v0/data/uberduck/steve-harvey/all.txt',
#             '/mnt/disk/uberduck-experiments-v0/data/uberduck/LJSpeech-1.1/metadata.csv',
#             '/mnt/disk/uberduck-experiments-v0/data/uberduck/LibriTTS/metadata_mellotron.txt',
#             '/mnt/disk/uberduck-experiments-v0/data/uberduck/vctk/metadata.txt']

# synthesize_speakerids(source_files) #need to change
# reindex()

# train,val  = get_testval_split()

# TTSModel('mellotron')

In [37]:
'''
Preprocessing Pipeline
1 - Load data
2 - Convert to correct sample rate using
3a - Detect silence on front
3b - Detect silence on back 
4a - Trim silence on front
4b - Trim silence on back
5 - Normalize mel spectrogram
6a - Pad silence to front
6b - Pad silence to back
'''
import librosa
import matplotlib.pyplot as plt
#import torchaudio
import torch
import numpy as np
import librosa.display
import glob
import pydub
from pydub import AudioSegment


class Loader:
    """ 
    Load audio file and resample to desired sample rate
        sample_rate (int): desired sample rate
    """

    def __init__(self, sample_rate=22050):
        self.sample_rate = sample_rate

    def load(self, file_path):
        signal = AudioSegment.from_file(file_path)
        signal = signal.set_frame_rate(self.sample_rate)
        return signal


class Padder:
    """
    Apply padding to array
        mode (str): argument for numpy padding 
        pad_time (int): numebr of ms to pad to left and right
    """

    def __init__(self, sample_rate, mode="constant", pad_ms=50):
        self.pad_time = pad_ms / 1000  # Convert to seconds
        self.sample_rate = sample_rate
        self.mode = mode

    def pad(self, arr):
        '''
        Pads zeros (silence) to front and back of audio signal
            ms: number of millseconds to bad for (ms)
            sr: sample_rate (Hz)
        '''

        n_padding = self.get_pad_length()
        padded_arr = self.left_pad(arr, n_padding)
        padded_arr = self.right_pad(padded_arr, n_padding)

        return padded_arr

    def get_pad_length(self):
        '''
        Calculates number of elements to pad to array
            sr (int): sample rate
            t (int): Number of seconds to pad for
        '''
        return int(self.sample_rate * self.pad_time)

    def left_pad(self, arr, n_missing_items):
        # [1,2,3] -> 2 -> [0,0,1,2,3]
        padded_array = np.pad(arr,
                              (n_missing_items, 0),
                              mode=self.mode)

        return padded_array

    def right_pad(self, arr, n_missing_items):
        # [1,2,3] -> 2 -> [1,2,3,0,0]
        padded_array = np.pad(arr,
                              (0, n_missing_items),
                              mode=self.mode)
        return padded_array


class TrimSilence():
    def __init__(self, mode="constant", leading_silence_threshold=-33, trailing_silence_threshold=-40):
        self.mode = mode
        self.leading_silence_threshold = leading_silence_threshold
        self.trailing_silence_threshold = trailing_silence_threshold

    def detect_silence(self, signal):
        '''
        Calls function to detect silence. 
        Could be with pydub, librosa, or pytorch VAD.
        '''
        start_trim, end_trim = self.pydub_detect_silence(signal)
        return start_trim, end_trim

    def crop_silence(self, signal, start_trim, end_trim):
        duration = len(signal)
        return signal[start_trim:duration-end_trim]

    # Calculate and return indices to trim for
    # leading and trailing silences
    def pydub_detect_silence(self, signal):
        start_trim = pydub.silence.detect_leading_silence(
            signal, silence_threshold=self.leading_silence_threshold)
        end_trim = pydub.silence.detect_leading_silence(
            signal.reverse(), silence_threshold=self.trailing_silence_threshold)
        return start_trim, end_trim

    # Main for trimming class
    def trim(self, signal):
        start_trim, end_trim = self.detect_silence(signal)
        signal_trimmed = self.crop_silence(signal, start_trim, end_trim)
        return signal_trimmed

# class LogSpectrogramExtractor:
#     """
#     Extracts log spectrograms (in dB) from a time series signal
#     frame_size
#     hop_length
#     """

#     def __init__(self, frame_size, hop_length):
#         self.frame_size = frame_size
#         self.hop_length = hop_length

#     def extract(self, signal):
#         # Returns (1+frame_size / 2, num_frames)
#         # remove last index to keep even
#         # 1024 -> 513 -> 512
#         stft = librosa.stft(signal,
#                             n_fft=self.frame_size,
#                             hop_length=self.hop_length)[:-1]
#         spectrogram = np.abs(stft)
#         log_spectrogram = librosa.amplitude_to_db(spectrogram)
#         return log_spectrogram


class Normalizer:
    """ 
    Apply min max normalization to an array. 
    Min value of array set to self.min, max value to self.max
    """

    def __init__(self, min_val, max_val):
        self.min = min_val
        self.max = max_val

    def librosa_normalize(self, signal):
        return librosa.util.normalize(signal)

    def min_max_normalize(self, arr):
        # Convert to range [0,1]
        norm_arr = (arr - arr.min() / (arr.max() - arr.min()))

        # Convert range to [self.min, self.max]
        norm_arr = norm_arr * (self.max - self.min) + self.min
        return norm_arr

    def min_max_denormalize(self, norm_arr, original_min, original_max):
        arr = (norm_arr - self.min) / (self.max - self.min)
        arr = arr * (original_max - original_min) + original_min

    def normalize(self, signal):
        return self.librosa_normalize(signal)


class PreprocessingPipeline:
    """ 
    Processes audio files in directory, applying the 
    following steps to each file:
    1 - load a file
    2 - pad the signal
    3 - extract log spectrogram fro msignal
    4 - normalize spectrogram
    5 - save normalized spectrogram
    Store min/max values for all log spectrograms
    """

    def __init__(self, loader=None, trimmer=None, normalizer=None, padder=None):
        self.loader = loader
        self.trimmer = trimmer
        self.normalizer = normalizer
        self.padder = padder

    def pydub_to_np(self, pydub_signal):
        return np.array(pydub_signal.get_array_of_samples(), dtype=np.float64)

    def save(self, signal, path, sr):
        librosa.output.write_wav(path, signal, sr=sr)

    # Main preprocessing function
    def preprocess(self, file_path):
        # Loads in pydub signal and converts to specified sample rate
        original_signal = self.loader.load(file_path)

        # Detects and trims leading and trailing silence
        processed_signal = self.trimmer.trim(original_signal)
        # print(trimmed_signal)

        # Convert pydub to numpy to use librosa norm
        processed_signal = self.pydub_to_np(processed_signal)
        # print(trimmed_signal_np)

        if self.normalizer:
            # Perform min/max normalization
            processed_signal = self.normalizer.normalize(processed_signal)
            # print(normalized_signal)

        # Pad signal with silence
        processed_signal = self.padder.pad(processed_signal)
        # print(padded_signal)
        return self.pydub_to_np(original_signal), processed_signal


In [44]:
import sys
#!{sys.executable} -m pip install pydub
!{sys.executable} -m pip install sounddevice